In [1]:
import pandas as pd
import numpy as np

import VFC_dyad
import VFC_tp_seg
import Idiff
import Idiff_bootstrap

### 输入输出和参数设置

In [2]:
# 输入数据路径
GROUP1_DATA2021_FOLDER = 'data2021/Class1-Parent'#亲子组
GROUP2_DATA2021_FOLDER = 'data2021/Class2-Stranger'#陌生人组
GROUP1_DATA2023_FOLDER = 'data2023/Class1-Parent'#亲子组
GROUP2_DATA2023_FOLDER = 'data2023/Class2-Stranger'#陌生人组
GROUP1_DATAALL_FOLDER = 'data_all/Class1-Parent'#亲子组
GROUP2_DATAALL_FOLDER = 'data_all/Class2-Stranger'#陌生人组

# 输入数据结构：
# 每个数据文件包含6种拼图场景，其中前面的1/2表示儿童拼或者成人拼，
# 后面的1/2/3表示单独/陪伴/合作,只关注3（合作）
sit_c1 = 'ca11' #儿童拼-单独
sit_a1 = 'ca21' #成人拼-单独
sit_c2 = 'ca12' #儿童拼-陪伴
sit_a2 = 'ca22' #成人拼-陪伴
sit_c3 = 'ca13' #儿童拼-合作
sit_a3 = 'ca23' #成人拼-合作
# 每种情况包含4个cell文件，分别对应4个频段：1-3，3-6，6-9，9-12
delta,theta,alpha,belta=0,1,2,3
freq_all=['delta','theta','alpha','belta']
# 每个cell文件的格式为通道数*时间点数*被试数
# 通道数: 其中1-28为儿童，29-56为成人，通过subc选择类型
# 28个通道及其对应的索引为：
channame_full = ['Fp1','Fp2','Fz','F3','F4','F7','F8','FC1','FC2','FC5','FC6','Cz','C3','C4','T7','T8','CP1','CP2','CP5','CP6','Pz','P3','P4','P7','P8','Oz','O1','O2']
id_chan_all = list(range(0,28))
# 26个通道及其对应的索引为：
channame_test = ['Fp1','Fp2','Fz','F3','F4','F7','F8','FC1','FC2','FC5','FC6','Cz','C3','C4','CP1','CP2','CP5','CP6','Pz','P3','P4','P7','P8','Oz','O1','O2']
id_chan_test = [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27]

In [6]:
# 参数设置
chan_num = 28 #通道数
edge_num = int(chan_num*(chan_num-1)/2) #边数
Nuptran_indices = VFC_dyad.uptran_mask(chan_num) #矩阵上三角列向量的位置
Euptran_indices = VFC_dyad.uptran_mask(edge_num) #矩阵上三角列向量的位置
Vtype='all'

# itype指边连接强度的计算方式，'p','c','d'分别代表：PLV复指数、corr相关、delta做差
itype_all=['e','c','d']
itype=itype_all[0]
# corr_type指边与边之间共波动特性的计算方式，'cos','pearson','delta'分别代表：
# 余弦相似度，皮尔逊相关系数，直接做差
corr_type_all=['cos','pearson','delta']
corr_type=corr_type_all[0]

# # tp_seg是将时间序列分段的基本单位，即一段时间序列的长度
# tp_seg_all = [2000,3000,4000]
# 步长设置为 100，总长度设置为 4000
tp_seg_all=range(2000,4100,100)
tp=2000

sit=sit_a3 #拼图场景
freq=alpha #频段
subc='adult'  #对象类型 

### 不同时间序列长度对应的结果

In [7]:
output_excel = 'output/Diff_mst_tpboot_all_'+subc+'-'+freq_all[freq]+'.xlsx'
# df=pd.DataFrame() #构造原始数据文件
# df.to_excel(output_excel)

In [8]:
import VeFC_mst
dyad_all = Idiff.file_path_infd(GROUP_DATAALL_FOLDER1)
edge_mst_idx = VeFC_mst.get_edge_mst_idx(dyad_all,sit,freq,subc,
                     corr_type,tp,edge_num)


In [12]:
for tp_seg in tp_seg_all:
    VeFC_mst_alld = VeFC_mst.VeFC_mst_all(dyad_all,sit,freq,subc,corr_type,tp_seg,
                             edge_mst_idx)
    VnFC_mode, trn_mode = VFC_tp_seg.Vnfc_dyad_seg(dyad_all,sit,freq,subc,Vtype,
                                                   Nuptran_indices,tp_seg)
    I_eFC_seg,I_nFC_seg = Idiff_bootstrap.Idiff_boot(res_time=1000,yita=0.3,VeFC_mode=VeFC_mst_alld,
                                                 VnFC_mode=VnFC_mode,trn_mode=trn_mode)
    
    # 转换为DataFrame格式
    I_eFC_allt=I_eFC_seg.T
    df_eFC1 = pd.DataFrame(I_eFC_allt)
    # 添加列名和索引
    df_eFC1.columns=['I_diff','I_self','I_other','Change_rate','Diff_rate'] #设置列名
    # 转换为DataFrame格式
    I_nFC_allt=I_nFC_seg.T
    df_nFC1 = pd.DataFrame(I_nFC_allt)
    # 添加列名和索引
    df_nFC1.columns=['I_diff','I_self','I_other','Change_rate','Diff_rate'] #设置列名

    # 创建一个新的工作表，命名为：场景+对象类型+频段
    new_sheet_name = str(tp_seg) 
    excel_writer = pd.ExcelWriter(output_excel, mode='a',engine='openpyxl')
    # 拼接
    df_all = pd.concat([df_eFC1,df_nFC1],axis=1)
    # 保存至excel
    df_all.to_excel(excel_writer, sheet_name=new_sheet_name)
    excel_writer.save()

d:\个人work\课题\亲子互动-功能连接\Code&Data\Idiff.py:46: RuntimeWarning: invalid value encountered in scalar divide
  I_selfi = (np.sum(R_mdi)-lenthi)/(lenthi*lenthi-lenthi)
d:\个人work\课题\亲子互动-功能连接\Code&Data\Idiff.py:57: RuntimeWarning: invalid value encountered in scalar divide
  I_selfi = (np.sum(R_mdi)-lenthi)/(lenthi*lenthi-lenthi)
C:\Users\吴诚成\AppData\Local\Temp\ipykernel_17444\2291937290.py:27: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  excel_writer.save()
d:\个人work\课题\亲子互动-功能连接\Code&Data\Idiff.py:46: RuntimeWarning: invalid value encountered in scalar divide
  I_selfi = (np.sum(R_mdi)-lenthi)/(lenthi*lenthi-lenthi)
d:\个人work\课题\亲子互动-功能连接\Code&Data\Idiff.py:57: RuntimeWarning: invalid value encountered in scalar divide
  I_selfi = (np.sum(R_mdi)-lenthi)/(lenthi*lenthi-lenthi)
C:\Users\吴诚成\AppData\Local\Temp\ipykernel_17444\2291937290.py:27: FutureWarning: save is not part of the public API, usage can give une

In [8]:
for tp_seg in tp_seg_all:
    df_read = pd.read_excel(output_excel,sheet_name=str(tp_seg))
    # 提取该时间长度eFC对应的Idiff值
    df_efc = df_read.iloc[:,1:2]
    df_efc.columns=[str(tp_seg)]
    # 提取该时间长度nFC对应的Idiff值
    df_nfc = df_read.iloc[:,6:7]
    df_nfc.columns=[str(tp_seg)]
    if tp_seg <= 100:
        df_efc_con = df_efc
        df_nfc_con = df_nfc
    else:
        df_efc_con = pd.concat([df_efc_con,df_efc],axis=1)
        df_nfc_con = pd.concat([df_nfc_con,df_nfc],axis=1)

In [9]:
# 创建一个新的工作表，命名为： efc_con
new_sheet_name = 'efc_con' 
excel_writer = pd.ExcelWriter(output_excel, mode='a',engine='openpyxl')
# 保存至excel
df_efc_con.to_excel(excel_writer, sheet_name=new_sheet_name)
excel_writer.save()

# 创建一个新的工作表，命名为： nfc_con
new_sheet_name = 'nfc_con' 
excel_writer = pd.ExcelWriter(output_excel, mode='a',engine='openpyxl')
# 保存至excel
df_nfc_con.to_excel(excel_writer, sheet_name=new_sheet_name)
excel_writer.save()

C:\Users\吴诚成\AppData\Local\Temp\ipykernel_7760\2891055729.py:6: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  excel_writer.save()
C:\Users\吴诚成\AppData\Local\Temp\ipykernel_7760\2891055729.py:13: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  excel_writer.save()
